In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random

from sklearn import decomposition

matplotlib_colors = ['b', 'g', 'r', 'c', 'm', 'grey', 'y', 'orange', 'pink', 'mediumspringgreen', 'darkcyan', 'darkviolet', 'darkslateblue', 'darkslategrey', 'skyblue', 'lime','indianred','maroon','darkolivegreen','dodgerblue'];

# Functions

Distance funcations

In [2]:
def distance(data_point, centroid, method):
    if method == "L1": 
            total_distance = np.linalg.norm(data_point-centroid,1)
    else:
            total_distance = np.linalg.norm(data_point-centroid,2)
    if method == "L2_sqr":
            total_distance = total_distance**2
    return total_distance;

Cluster Plotting

In [3]:
def cluster_plot(title, labels, colors, data):
    # Generates Cluster Graph
    plt.title(title, {'fontsize':28})
    plt.rcParams["figure.figsize"] = (20,20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    cols = [colors[p] for p in labels]
    plt.scatter(data[:,0], data[:,1], s=50, c=cols)
    plt.show()
    return;

Normalizing or standardizing data

In [4]:
def normalize_set(data_set):
    data = np.copy(data_set)
    mean_value = np.mean(data, axis = 0)
    std_value = np.std(data, axis = 0)
    data = (data-mean_value)/std_value
    return data

Kmeans and Kmeans++ algorithm

In [5]:
def AS_kmeans(data_set, num_clusters,method = 'random', dist_func = 'L2'):
    data = normalize_set(data_set)
    PCA = decomposition.PCA(n_components = 2)
    data = PCA.fit_transform(data)
    
    centers = [0]*num_clusters
    error = 99
    labels = [0]*len(data)
    dist_centroids = [999999999]*len(data)
    count = 1
    try:
        del centroids
    except NameError:
        None
    
    #Initiate Centroids with kmeans or kmeans++ process
    for x in range(num_clusters):
        center = random.randint(0,len(data)-1)
        try:
            centroids
        except NameError:
            centroids = [data[center]]
            new_centroid_prob = np.ones(len(data))*9999999
            centers[x] = center
        else:
            if method == "K-means++":
                print('K-means++')
                for y in centroids:
                    for z in range(len(data)):
                        new_centroid_prob[z] = min(new_centroid_prob[z],np.linalg.norm(data[z]-y,2)**2)
                distance_sum = new_centroid_prob.sum()
                new_centroid_prob = new_centroid_prob/distance_sum
                next_centroid = np.random.choice(np.arange(len(data)), 1, p=new_centroid_prob)[0]
                centroids = np.vstack((centroids,data[next_centroid]))
            else:
                while center in centers: 
                    center  = random.randint(0,len(data)-1)
                centroids = np.vstack((centroids,data[center]))
                centers[x] = center
    del center
    del centers 
    del new_centroid_prob
    prev_centroids = -centroids

    #Running kmeans process
    while error > 0.01:
        count += 1
        for i in range(len(data)):
            dist_centroid = 999999999;
            for j in range(len(centroids)):
                if method == "K-Median":
                    k_distance = distance(data[i], centroids[j], 'L1')
                else:
                    k_distance = distance(data[i], centroids[j], dist_func)
                    
                if k_distance < dist_centroid:
                    labels[i] = j
                    dist_centroid = k_distance
                    #dist_centroids[i] = k_distance
                    dist_centroids[i] = distance(data[i], centroids[j], 'L2_sqr')

        error = np.max(abs(abs(prev_centroids-centroids)/centroids))
        if error > 0.01:
            data_df = pd.DataFrame(labels , columns = ["labels"])
            for col in range(len(data.T)):
                col_name = 'x'+str(col);
                data_df[col_name] = data[:,col].tolist()

            prev_centroids = centroids.copy()
            for i in range(len(centroids)):
                for j in range(len(centroids[0])):
                    centroids[i,j] = data_df[data_df['labels'] == i].mean()[j+1]
        if count == 100000:
            print(count)
            break
            
    print("number of iterations: ", count)
    #print(centroids)
    return labels, data, sum(dist_centroids);